In [1]:
import cv2 #openCV
import exiftool
import os, glob
import numpy as np
import pyzbar.pyzbar as pyzbar
import matplotlib.pyplot as plt
import mapboxgl

print()
print("Successfully imported all required libraries.")
print()

if os.name == 'nt':
    if os.environ.get('exiftoolpath') is None:
        print("Set the `exiftoolpath` environment variable as described above")
    else:
        if not os.path.isfile(os.environ.get('exiftoolpath')):
            print("The provided exiftoolpath isn't a file, check the settings")

try:
    with exiftool.ExifTool(os.environ.get('exiftoolpath')) as exift:
        print('Successfully executed exiftool.')
except Exception as e:
    print("Exiftool isn't working. Double check that you've followed the instructions above.")
    print("The execption text below may help to find the source of the problem:")
    print()
    print(e)


Successfully imported all required libraries.

Successfully executed exiftool.


In [ ]:
from micasense.image import Image
repo = 'REDEDGE-MX' # ALTUM, ALTUM-PT, REDEDGE-P, 0000SET
imagePath = os.path.join('.','data','0000SET', '000')
imageName = glob.glob(os.path.join(imagePath,'IMG_0000_1.tif'))[0]

img = Image(imageName)
img.plot_raw(figsize=(8.73,8.73))

from micasense.panel import Panel
panel = Panel(img)
if not panel.panel_detected():
    raise IOError("Panel Not Detected! Check your installation of pyzbar")
else:
    panel.plot(figsize=(8,8))

print('Success! Now you are ready for Part 1 of the tutorial.')

In [ ]:
import requests
import time

# Adresse IP de la caméra
CAMERA_IP = "192.168.1.83"
CAPTURE_URL = f"http://{CAMERA_IP}/capture"
CAPTURE_CONF_URL = f"http://{CAMERA_IP}/config"
DETECT_PANNEL_URL = f"http://{CAMERA_IP}/detect_panel"

class TeddyFlight:
    def __init__(self, capture_interval:float = 0.9) -> None:
        self.capture_interval = capture_interval
        pass
    
    def detect_pannel(self):
        cresponse = self.capture_image(args={"detect_panel": True})
        if cresponse.get("status") == "error":
            print("Camera failed to detect a pannel\nRetrying in 3sRetrying in 3s............")
            time.sleep(3)
            self.detect_pannel()

    def set_config(self):
        capture_conf = {"auto_cap_mode": "timer", "timer_period": self.capture_interval, "audio_enable": True}
        response = requests.post(CAPTURE_URL, json=capture_conf)
        if response.status_code == 200:
            print("Configuration set successfully")
        else:
            print("Failed to configure")
            
    def capture_image(self, args:dict = {}):
        presponse = requests.post(DETECT_PANNEL_URL, json={"abort_detect_panel": True})
        if presponse.status_code == 200:
            assert presponse.json().get("detect_panel") == False, "Failed to abort the pannel detection mode even though the request was successful\nProbably an internal error"
        else:
            print("Failed to abort the pannel detection mode")
            self.capture_image()
            
        if args == {}:
            response = requests.get(CAPTURE_URL)
        else:
            response = requests.post(CAPTURE_URL, json=args)
        if response.status_code == 200:
            print("Image captured successfully")
        else:
            print("Failed to capture image")
        return response.json()
    
    def launch(self):
        self.detect_pannel()
        self.set_config()
        self.capture_image()
        
        # The following approach offers more flexibility on each capture but may increase the delta time between each capture
        #while True:
        #    self.capture_image()
        #    time.sleep(self.capture_interval)

if __name__ == "__main__":
    TeddyFlight().launch()
